In [2]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute, extensions
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
# provider = IBMQ.load_account()

Hamiltonian describes the situation in a physical system. For the sensing application we choose a Hamiltonian, $H= \sigma_{z}$. We will do the time evolution of our hamiltonian for pi/3 time.

In [3]:
circ = QuantumCircuit(2,2)
def matrix(qc):
    backend = Aer.get_backend('unitary_simulator')
    job =  execute(qc, backend)
    result = job.result()
    return result
circ.z(0)
circ.z(1)
mat = matrix(circ).get_unitary(circ, decimals= 3)
print(mat)


Operator([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
          [-0.+0.j, -1.+0.j, -0.+0.j, -0.+0.j],
          [-0.+0.j, -0.+0.j, -1.+0.j, -0.+0.j],
          [ 0.-0.j,  0.-0.j,  0.-0.j,  1.-0.j]],
         input_dims=(2, 2), output_dims=(2, 2))


Following cell wokr 

In [4]:
def hamilton(operator, time):
    gate = extensions.HamiltonianGate(operator, time)
    return gate

evolution = hamilton(mat, np.pi/3)
print(evolution)
#ham_gate = QuantumCircuit(2)
#ham_gate.unitary(evolution, [0,1])
circuit = QuantumCircuit(2,2)
circuit.append(evolution, [0, 1])
display(circuit.draw())

tran_ham = transpile(circuit, basis_gates=['cx','u3'])
display(tran_ham.draw())


Instruction(name='hamiltonian', num_qubits=2, num_clbits=0, params=[array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [-0.+0.j, -1.+0.j, -0.+0.j, -0.+0.j],
       [-0.+0.j, -0.+0.j, -1.+0.j, -0.+0.j],
       [ 0.-0.j,  0.-0.j,  0.-0.j,  1.-0.j]]), 1.0471975511965976])


┌──────────────┐
q_0: ┤0             ├
     │  Hamiltonian │
q_1: ┤1             ├
     └──────────────┘
c: 2/════════════════

global phase: 5.7818
         ┌──────────────┐              ┌───────────────┐           »
q_0: ────┤ U3(0,-π,π/2) ├──────■───────┤ U3(π,-π,-π/2) ├────────■──»
     ┌───┴──────────────┴───┐┌─┴─┐┌────┴───────────────┴─────┐┌─┴─┐»
q_1: ┤ U3(0.067571,-π/2,-π) ├┤ X ├┤ U3(1.5371,1.5123,-2.615) ├┤ X ├»
     └──────────────────────┘└───┘└──────────────────────────┘└───┘»
c: 2/══════════════════════════════════════════════════════════════»
                                                                   »
«     ┌──────────────────────┐
«q_0: ┤ U3(π,1.0528,-2.0888) ├
«     └┬────────────────────┬┘
«q_1: ─┤ U3(1.6384,-π,-π/2) ├─
«      └────────────────────┘ 
«c: 2/════════════════════════
«

In [5]:
bell= QuantumCircuit(2,2)
bell.h(0)
bell.cx(0,1)
bell.append(tran_ham, [0,1])
bell.cx(0,1)
bell.h(0)
display(bell.draw())
bell.measure_all()
backend = Aer.get_backend('qasm_simulator')
job = execute(bell, backend, shots = 1024)
result_sim = job_sim.result()
counts = result_sim.get_counts(bell)
print(counts)
from qiskit.visualization import plot_histogram
plot_histogram(counts)


┌───┐     ┌──────────────┐     ┌───┐
q_0: ┤ H ├──■──┤0             ├──■──┤ H ├
     └───┘┌─┴─┐│  circuit-142 │┌─┴─┐└───┘
q_1: ─────┤ X ├┤1             ├┤ X ├─────
          └───┘└──────────────┘└───┘     
c: 2/════════════════════════════════════

DAGCircuitError: 'expected 2 wires, got 4'